In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pickle
from IPython.core.display import display, HTML
import pandas as pd

import tensorflow as tf
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from keras import backend as K

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sys import path
path.append(".."); from generate_patients import Patient

In [ ]:
display(HTML("<style>.container { width:90% !important; }</style>"))
plt.interactive(True)
%matplotlib inline
plt.rcParams['figure.figsize'] = [28, 6]

In [ ]:
df_results = pd.read_pickle("./results.pkl")  # Load pandas dataframe containing results summary
df_results

In [ ]:
df_results = df_results.drop(index='1088F_128C_SVM')

In [ ]:
df_results.to_pickle("./results.pkl")

In [ ]:
column_names = ['Signal', 'Method', 'Features', 'Components', 'AUC_Score', 'F1_Score', 'Average_Precision', 'Balanced_Accuracy']
results = pd.DataFrame(columns=column_names)
results

In [ ]:
(results.loc[index_name, 'AUC_Score'] + results.loc[index_name, 'F1_Score'])/2

In [ ]:
index_name = '1088F_64Ca_SVM'

if index_name not in results.index or 2>3:
    print('aeee')
    row_values = ['Whole', 'SVM', '1088', '64', 0.941176, 0.881768, 0.861029, 0.767123]
    results.loc[index_name, :] = row_values
    results

In [ ]:
value = results.loc[index_name, 'AUC_Score'] + results.loc[index_name, 'F1_Score']

In [ ]:
# index_name = '1088F_64C_SVM'
# if (auc_score + F1_score)/2 > results.loc[index_name, 'AUC_Score'] + results.loc[index_name, 'F1_Score']:
#     results.loc[index_name, :] = [auc_score, average_precision, balanced_accuracy, F1_score]
#     plt.savefig(index_name + '.png', bbox_inches='tight')

In [ ]:
with open('../patients.obj', 'rb') as patients_file:
    patients = pickle.load(patients_file)
    
print(len(patients))

In [ ]:
valid_patients = [patient for patient in patients if patient.features[10] >= 0]
print('Number of patients: %s' % len(valid_patients))

In [ ]:
df = pd.DataFrame([patient.features for patient in valid_patients],
                   columns=['ID', 'pse', 'psp1', 'psp2', 'psp3', 'pspf1', 'pspf2', 'pspf3', 'wpsp', 'cpt', 'faller', 'faller_3M', 'faller_6M', 'faller_9M', 'faller_12M', 'age'])

# Column formating
df['ID'] = df['ID'].apply(lambda value: str(int(value)))
df['age'] = df['age'].apply(lambda value: int(value))

faller_columns = ['faller', 'faller_3M', 'faller_6M', 'faller_9M', 'faller_12M']
df[faller_columns] = df[faller_columns].applymap(lambda value: int(value))

round(df.describe(), 3)

In [ ]:
df.loc[df['faller'] == True].head()

In [ ]:
df.loc[df['faller'] == False].head()

In [ ]:
pd.concat([df.loc[df['faller'] == True], df.loc[df['faller'] == False]], axis=1, keys=['👎 Faller', '👍 Non-Faller']).describe()

### Seaborn

In [ ]:
import seaborn as sns

In [ ]:
plt.subplot(141)
plt.subplot(142)
plt.subplot(143)
plt.subplot(212)
sns.barplot(['AUC_Score', 'F1_Score'], [results.loc[index_name, 'AUC_Score'], results.loc[index_name, 'F1_Score']])
# plt.tight()

In [ ]:
plt.subplot(221)
sns.barplot(x=['AUC_Score', 'F1_Score'], y=[4, 3])

plt.subplot(222)
sns.barplot(x=['AUC_Score', 'F1_Score'], y=[4, 3])

plt.subplot(212)
sns.barplot(x=['AUC_Score', 'F1_Score'], y=[4, 3])

plt.show()

In [ ]:
corr = df.corr().iloc[:32, :32]

fig, (ax) = plt.subplots(1, 1, figsize=(24,12))
hm = sns.heatmap(corr, 
                 ax=ax,           # Axes in which to draw the plot, otherwise use the currently-active Axes.
                 cmap="coolwarm", # Color Map.
                 #square=True,    # If True, set the Axes aspect to “equal” so each cell will be square-shaped.
                 annot=True, 
                 fmt='.2f',       # String formatting code to use when adding annotations.
                 #annot_kws={"size": 14},
                 linewidths=.05)

fig.subplots_adjust(top=0.93)
fig.suptitle('Features Correlation Heatmap', fontsize=14, fontweight='bold')

In [ ]:
sns.pairplot(df, vars=components_list[:8], hue='faller')

In [ ]:
sns.pairplot(df, vars=['pse', 'psp1', 'psp2', 'psp3', 'pspf1', 'pspf2', 'pspf3', 'wpsp', 'cpt'], hue='faller')

In [ ]:
sns.countplot(data=df, x='age', hue="faller")
sns.jointplot(data=df, x='faller', y='faller_3M', kind='kde', space=0, height=6, ratio=4)

### Grid Search and Classifier Testing

In [ ]:
# pca = PCA(n_components=64)
# X_train = pca.fit_transform(features_norm_train)
# X_test = pca.transform(features_norm_test)
# print('PCA Explained Variance:', sum(pca.explained_variance_ratio_))

# grid_search = GridSearchCV(next(adab_classifier), param_grid=adab_parameter_grid, cv=StratifiedKFold(n_splits=4), scoring='roc_auc', n_jobs=-1)
# grid_search.fit(X_train, y_train)
# print('Best score: %s' % grid_search.best_score_)
# print('Best parameters: %s\n' % grid_search.best_params_)

# pd.set_option('max_colwidth',120)
# pd.DataFrame(grid_search.cv_results_)[['mean_test_score', 'params']].sort_values('mean_test_score', ascending=False)

# # collect_and_print_results(grid_search.best_estimator_.predict_proba(X_test)[:, 1])